##                                   BUAN 6341.002 Project 3
              
   ##                         Group 2 : Sumanth Appalakutti, Jaahnavi Tiruthani                                                       

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import precision_recall_fscore_support

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [4]:
data = pd.read_csv('ENB2012_data.csv')

In [5]:
data.columns = ["relative_compactness", "surface_area", "wall_area",
              "roof_area", "overall_height", "orientation", "glazing_area",
             "glazing_area_distribution", "heating_load", "cooling_load"]

In [6]:
data.head()

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [7]:
X = data.iloc[:,:8]
Y1 = data.iloc[:,-2]
Y2 = data.iloc[:,-1]

In [8]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X,Y1,random_state=10)
X2_train, X2_test, y2_train, y2_test = train_test_split(X,Y2,random_state=10)

In [9]:
X.shape, Y1.shape, Y2.shape

((768, 8), (768,), (768,))

In [10]:
scaler = MinMaxScaler()
X1_train_scaled = scaler.fit_transform(X1_train)
X1_test_scaled = scaler.transform(X1_test)
X2_train_scaled = scaler.fit_transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)

In [11]:
# create model
model = Sequential()
model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu')) # hidden layer
model.add(Dense(10, kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer='normal'))

In [12]:
# Compile model
model.compile(loss='mse', optimizer='adam' , metrics = ['mse','mae'])

In [13]:
model.fit(X1_train_scaled, y1_train, epochs = 100, batch_size = 20)

Epoch 1/100
576/576 [==============================] - 2s 4ms/step - loss: 614.0119 - mean_squared_error: 614.0119 - mean_absolute_error: 22.6140
Epoch 2/100
576/576 [==============================] - 0s 174us/step - loss: 611.2748 - mean_squared_error: 611.2748 - mean_absolute_error: 22.5536
Epoch 3/100
576/576 [==============================] - 0s 136us/step 

In [14]:
model.fit(X2_train_scaled, y2_train, epochs = 100, batch_size = 20)

Epoch 1/100
576/576 [==============================] - 0s 92us/step - loss: 15.7414 - mean_squared_error: 15.7414 - mean_absolute_error: 3.1141
Epoch 2/100
576/576 [==============================] - 0s 109us/step - loss: 13.8978 - mean_squared_error: 13.8978 - mean_absolute_error: 2.9268
Epoch 3/100
576/576 [==============================] - 0s 92us/step - loss: 13.5214 - mean_squared_error: 13.5214 - mean_absolute_error: 2.8086
Epoch 4/100
576/576 [==============================] - 0s 142us/step - loss: 13.381

In [15]:
from sklearn.metrics import r2_score

y1_train_predict = model.predict(X1_train_scaled)
y1_test_predict = model.predict(X1_test_scaled)
y2_train_predict = model.predict(X2_train_scaled)
y2_test_predict = model.predict(X2_test_scaled)

print('Train r2: {:.2f}'.format(r2_score(y1_train, y1_train_predict)))
print('Test r2: {:.2f}'.format(r2_score(y1_test, y1_test_predict)))

print('Train r2: {:.2f}'.format(r2_score(y2_train, y2_train_predict)))
print('Test r2: {:.2f}'.format(r2_score(y2_test, y2_test_predict)))

Train r2: 0.83
Test r2: 0.82
Train r2: 0.87
Test r2: 0.89


In [16]:
knnreg=KNeighborsRegressor()
# Considering a range of k values
k_value={'n_neighbors':range(1,50)}
# finding the best k value using Grid Search
gridk1=GridSearchCV(knnreg,param_grid=k_value, cv=10)
# Running the KNN regressor with best parameter on the training data
knnmodely1 = gridk1.fit(X1_train_scaled,y1_train)
print("R-Square of K-NN regressor on training data:", '%.4f' %knnmodely1.best_score_)
print("R-Square of K-NN regressor on test data:",'%4f'%knnmodely1.score(X1_test_scaled, y1_test))
print("Best Parameters:")
print(knnmodely1.best_params_)
print("Best Estimator:")
print(knnmodely1.best_estimator_)

scaledLR=LinearRegression().fit(X1_train_scaled,y1_train)
print("The R-Square of Linear Regression on training data is : ",'%.4f' %scaledLR.score(X1_train_scaled,y1_train))
print("The R-Square of Linear Regression on test data is: ",'%.4f' %scaledLR.score(X1_test_scaled,y1_test))
print("Average Cross Validation Score of Linear Regression on Training Data:",
      '%.4f' %cross_val_score(LinearRegression(), X1_train_scaled, y1_train, cv = 10).mean())

scaledlassoLR=Lasso()
param={'alpha':[0.1,1,10,50,100], 'max_iter':[100,1000,10000,100000]}
gridlasso=GridSearchCV(scaledlassoLR,param_grid=param, cv=10)
lassomodel = gridlasso.fit(X1_train_scaled,y1_train)
#Printing the Accuracy of Training and testing data
print("The R-Square of Lasso on Training data is : ",'%.4f' %lassomodel.best_score_)
print("The R-Square of Lasso on Test data is : ",'%.4f' %lassomodel.best_score_)
# Printing the Best parameters and Best estimator
print("Best Parameters:")
print(gridlasso.best_params_)
print("Best Estimator:")
print(gridlasso.best_estimator_)

scaledridge=Ridge()
alpha={'alpha':[0.1,1,5,10,15,20,25], 'max_iter':[100,1000,10000,100000]}
# Finding the best parameters for Ridge using Grid Search
gridridge=GridSearchCV(scaledridge,param_grid=alpha, cv=10)
# Fitting the model
ridgemodel = gridridge.fit(X1_train_scaled,y1_train)
print("The R-Square of Lasso on Training data is : ",'%.4f' %ridgemodel.best_score_)
print("The R-Square of Lasso on Test data is : ",'%.4f' %ridgemodel.best_score_)
print("Best Parameters:")
print(gridridge.best_params_)
print("Best Estimator:")
print(gridridge.best_estimator_)

R-Square of K-NN regressor on training data: 0.9223
R-Square of K-NN regressor on test data: 0.944745
Best Parameters:
{'n_neighbors': 10}
Best Estimator:
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')
The R-Square of Linear Regression on training data is :  0.9119
The R-Square of Linear Regression on test data is:  0.9285
Average Cross Validation Score of Linear Regression on Training Data: 0.9083
The R-Square of Lasso on Training data is :  0.9017
The R-Square of Lasso on Test data is :  0.9017
Best Parameters:
{'alpha': 0.1, 'max_iter': 100}
Best Estimator:
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=100,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
The R-Square of Lasso on Training data is :  0.9083
The R-Square of Lasso on Test data is :  0.9083
Best Parameters:
{'alpha': 0

In [17]:
knnreg=KNeighborsRegressor()
# Considering a range of k values
k_value={'n_neighbors':range(1,50)}
# finding the best k value using Grid Search
gridk2=GridSearchCV(knnreg,param_grid=k_value, cv=10)
# Running the KNN regressor with best parameter on the training data
knnmodely2 = gridk2.fit(X2_train_scaled,y2_train)
print("R-Square of K-NN regressor on training data:", '%.4f' %knnmodely2.score(X2_train_scaled, y2_train))
print("R-Square of K-NN regressor on test data:",'%4f' %knnmodely2.score(X2_test_scaled, y2_test))
print("Best Parameters:")
print(knnmodely2.best_params_)
print("Best Estimator:")
print(knnmodely2.best_estimator_)

scaledLR=LinearRegression().fit(X2_train_scaled,y2_train)
print("The R-Square of Linear Regression on training data is : ",'%.4f' %scaledLR.score(X2_train_scaled,y2_train))
print("The R-Square of Linear Regression on test data is: ",'%.4f' %scaledLR.score(X2_test_scaled,y2_test))
print("Average Cross Validation Score of Linear Regression on Training Data:",
      '%.4f' %cross_val_score(LinearRegression(), X2_train_scaled, y2_train, cv = 10).mean())

scaledlassoLR=Lasso()
param={'alpha':[0.1,1,10,50,100], 'max_iter':[100,1000,10000,100000]}
gridlasso=GridSearchCV(scaledlassoLR,param_grid=param, cv=10)
lassomodel = gridlasso.fit(X2_train_scaled,y2_train)
#Printing the Accuracy of Training and testing data
print("The R-Square of Lasso on Training data is : ",'%.4f' %lassomodel.best_score_)
print("The R-Square of Lasso on Test data is : ",'%.4f' %lassomodel.best_score_)
# Printing the Best parameters and Best estimator
print("Best Parameters:")
print(gridlasso.best_params_)
print("Best Estimator:")
print(gridlasso.best_estimator_)

scaledridge=Ridge()
alpha={'alpha':[0.1,1,5,10,15,20,25], 'max_iter':[100,1000,10000,100000]}
# Finding the best parameters for Ridge using Grid Search
gridridge=GridSearchCV(scaledridge,param_grid=alpha, cv=10)
# Fitting the model
ridgemodel = gridridge.fit(X2_train_scaled,y2_train)
print("The R-Square of Lasso on Training data is : ",'%.4f' %ridgemodel.best_score_)
print("The R-Square of Lasso on Test data is : ",'%.4f' %ridgemodel.best_score_)
print("Best Parameters:")
print(gridridge.best_params_)
print("Best Estimator:")
print(gridridge.best_estimator_)

R-Square of K-NN regressor on training data: 0.9189
R-Square of K-NN regressor on test data: 0.934966
Best Parameters:
{'n_neighbors': 10}
Best Estimator:
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')
The R-Square of Linear Regression on training data is :  0.8802
The R-Square of Linear Regression on test data is:  0.9093
Average Cross Validation Score of Linear Regression on Training Data: 0.8766
The R-Square of Lasso on Training data is :  0.8668
The R-Square of Lasso on Test data is :  0.8668
Best Parameters:
{'alpha': 0.1, 'max_iter': 100}
Best Estimator:
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=100,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
The R-Square of Lasso on Training data is :  0.8765
The R-Square of Lasso on Test data is :  0.8765
Best Parameters:
{'alpha': 0

In [18]:
knny = KNeighborsRegressor(n_neighbors=10)
knny1 = knny.fit(X1_train_scaled,y1_train)
knny2 = knny.fit(X2_train_scaled,y2_train)
knnby1 = knny1.predict(X1_test_scaled)
knnby2 = knny2.predict(X2_test_scaled)

In [19]:
knnby = knnby1 + knnby2

In [20]:
knnby.shape

(192,)

In [21]:
np.percentile(knnby, [0,25,50,75,100])

array([25.372 , 32.675 , 37.838 , 63.9095, 78.042 ])

In [22]:
knny = KNeighborsRegressor(n_neighbors=10)
knny1 = knny.fit(X,Y1)
knny2 = knny.fit(X,Y2)
knnby1 = knny1.predict(X)
knnby2 = knny2.predict(X)
knnby = knnby1 + knnby2

In [23]:
np.percentile(knnby, [0,25,50,75,100])

array([27.374 , 30.933 , 47.059 , 65.0465, 86.748 ])

In [24]:
y2 =pd.DataFrame(knnby, columns=['YP'], dtype='Float64')

In [25]:
X['YP']= y2

In [26]:
X

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,YP
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,53.814
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,53.094
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,52.896
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,53.450
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,63.820
5,0.90,563.5,318.5,122.50,7.0,3,0.0,0,61.914
6,0.90,563.5,318.5,122.50,7.0,4,0.0,0,62.660
7,0.90,563.5,318.5,122.50,7.0,5,0.0,0,65.090
8,0.86,588.0,294.0,147.00,7.0,2,0.0,0,57.880
9,0.86,588.0,294.0,147.00,7.0,3,0.0,0,55.930


In [27]:
X['label'] = np.where(X['YP']<=32, 1, np.where(X['YP']<=59, 2, 3))

In [28]:
X['label'].value_counts()

1    279
3    271
2    218
Name: label, dtype: int64

In [29]:
np.percentile(X['YP'], [0,10,20,30,40,50,60,70,80,90,100])

array([27.374 , 29.8842, 30.6464, 31.3648, 32.8972, 47.059 , 57.678 ,
       62.1486, 68.7404, 74.4746, 86.748 ])

In [30]:
X2 = X.drop(['YP'],axis=1)

In [31]:
X2.head()

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,label
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,2
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,2
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,2
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,2
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,3


In [57]:
XC = X2.iloc[:,:8]
YC = X2.iloc[:,-1]

from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(XC,YC,random_state=10)

In [61]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier

encoder = LabelEncoder()
encoder.fit(YC)
Ye = encoder.transform(YC)
Yd = np_utils.to_categorical(Ye)

X1_train, X1_test, y1_train, y1_test = train_test_split(XC,Yd, random_state=10)

np.random.seed(10)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X1_train)
X_test_scaled = scaler.transform(X1_test)


In [62]:
X1_train.head()

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution
235,0.64,784.0,343.0,220.50,3.5,5,0.10,4
576,0.98,514.5,294.0,110.25,7.0,2,0.40,2
22,0.76,661.5,416.5,122.50,7.0,4,0.00,0
451,0.79,637.0,343.0,147.00,7.0,5,0.25,4
616,0.64,784.0,343.0,220.50,3.5,2,0.40,2


In [70]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu')) #hidden layer
model.add(Dense(3, activation='softmax')) #output layer

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [71]:
# Fit the model
model.fit(X_train_scaled, y1_train, epochs=150, batch_size=10)

Epoch 1/150
576/576 [==============================] - 1s 2ms/step - loss: 0.6052 - acc: 0.6788
Epoch 2/150
576/576 [==============================] - 0s 287us/step - loss: 0.5483 - acc: 0.7245
Epoch 3/150
576/576 [==============================] - 0s 252us/step - loss: 0.5186 - acc: 0.7326

In [72]:
# evaluate the model
scores = model.evaluate(X_test_scaled, y1_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 1ms/step

acc: 94.79%


In [73]:
np.random.seed(10)
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu')) #hidden layer
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for epochs in [100,200,300,350]:
    for batch_size in [5,10,15,20,25]:
        seed = 10
        np.random.seed(seed)
        model.fit(X_train_scaled, y1_train, epochs=epochs, batch_size=batch_size, verbose=0)

        scores_train = model.evaluate(X_train_scaled, y1_train)
        scores_test = model.evaluate(X_test_scaled, y1_test)

        print('\n')
        print('Batch size: ',end="")
        print(batch_size)
        print('epochs: ',end="")
        print(epochs)
        print('Training ',end="")
        print("%s: %.2f%%" % (model.metrics_names[1], scores_train[1]*100))
        print('Testing ',end="")
        print("%s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))

192/192 [==============================] - 0s 63us/step


Batch size: 5
epochs: 100
Training acc: 91.84%
Testing acc: 90.10%
192/192 [==============================] - 0s 68us/step


Batch size: 10
epochs: 100
Training acc: 95.14%
Testing acc: 94.79%
192/192 [==============================] - 0s 73us/step


Batch size: 15
epochs: 100
Training acc: 95.49%
Testing acc: 95.31%
192/192 [==============================] - 0s 68us/step


Batch size: 20
epochs: 100
Training acc: 95.49%
Testing acc: 94.79%
192/192 [==============================] - 0s 73us/step


Batch size: 25
epochs: 100
Training acc: 95.66%
Testing acc: 95.31%
192/192 [==============================] - 0s 78us/step


Batch size: 5
epochs: 200
Training acc: 95.

Epoch 300 Batch size =25

In [75]:
np.random.seed(10)
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu')) #hidden layer
model.add(Dense(3, activation='softmax'))

for optim in ['SGD', 'RMSprop', 'Adagrad', 'Adam']:
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    seed = 10
    np.random.seed(seed)
    model.fit(X_train_scaled, y1_train, epochs=300, batch_size=25, verbose=0)

    scores_train = model.evaluate(X_train_scaled, y1_train)
    scores_test = model.evaluate(X_test_scaled, y1_test)

    print('\n')
    print('Optimizer: ',end="")
    print(optim)
    print('Training ', end="")
    print("%s: %.2f%%" % (model.metrics_names[1], scores_train[1]*100))
    print('Testing ', end="")
    print("%s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))

192/192 [==============================] - 0s 73us/step


Optimizer: SGD
Training acc: 68.06%
Testing acc: 74.48%
192/192 [==============================] - 0s 89us/step


Optimizer: RMSprop
Training acc: 73.61%
Testing acc: 80.21%
192/192 [==============================] - 0s 115us/step


Optimizer: Adagrad
Training acc: 77.78%
Testing acc: 82.81%
192/192 [==============================] - 0s 177us/step


Optimizer: Adam
Training acc: 94.97%
Testing acc: 96.35%


Adam is the Best Optimizer.

In [78]:
for activation in ['softmax', 'relu', 'tanh', 'sigmoid', 'linear']:
    
    np.random.seed(10)
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation=activation))
    model.add(Dense(8, activation='relu')) #hidden layer
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    seed = 10
    np.random.seed(seed)
    model.fit(X_train_scaled, y1_train, epochs=300, batch_size=25, verbose=0)

    scores_train = model.evaluate(X_train_scaled, y1_train)
    scores_test = model.evaluate(X_test_scaled, y1_test)

    print('\n')
    print('activation: ',end="")
    print(activation)
    print('Training ', end="")
    print("%s: %.2f%%" % (model.metrics_names[1], scores_train[1]*100))
    print('Testing ', end="")
    print("%s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))

192/192 [==============================] - 0s 136us/step


activation: softmax
Training acc: 83.68%
Testing acc: 83.33%
192/192 [==============================] - 0s 68us/step


activation: relu
Training acc: 83.85%
Testing acc: 87.50%
192/192 [==============================] - 0s 109us/step


activation: tanh
Training acc: 94.79%
Testing acc: 95.83%
192/192 [==============================] - 0s 89us/step


activation: sigmoid
Training acc: 71.35%
Testing acc: 72.40%
192/192 [==============================] - 0s 83us/step


activation: linear
Training acc: 76.04%
Testing acc: 78.65%


best activation for input layer is tanh

In [86]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='tanh'))
model.add(Dense(8, activation='relu')) #hidden layer
model.add(Dense(3, activation='softmax')) #output layer
seed = 10
np.random.seed(seed)
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train_scaled, y1_train, epochs=300, batch_size=25)

Epoch 1/300
576/576 [==============================] - 2s 3ms/step - loss: 0.7855 - acc: 0.5538
Epoch 2/300
576/576 [==============================] - 0s 195us/step - loss: 0.6602 - acc: 0.6192
Epoch 3/300
576/576 [==============================] - 0s 148us/step - loss: 0.6167 - acc: 0.6487
Epoch 4/300
576/576 [==============================] - 0s 136us/step - loss: 0.5941 - acc: 0.6696

In [87]:
# evaluate the model
scores = model.evaluate(X_test_scaled, y1_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 1s 3ms/step

acc: 96.18%


# The best NN classification parameters as per our investigation provides a 96% accuracy. THe parameters are Optimizer = Adam, Input layer activation of Tanh, Epoch =300 and Batch Size = 25. Further investigation can be done by changing the number of Hidden layers, the nodes per layer, and activation changed per each layer, learning rate etc.